<a href="https://colab.research.google.com/github/anastaszi/255_datamining/blob/main/HW4_Anastasia_Zimina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approximate Nearest Neighbors
* LSH
* Exhaustive search
* Product Quantization
* Trees and Graphs
* HNSW

# Library Imports

In [1]:
!pip install datasketch

     |████████████████████████████████| 67 kB 4.3 MB/s 


In [2]:
!python -m pip install --upgrade faiss-gpu

     |████████████████████████████████| 89.7 MB 14 kB/s 


In [4]:
!pip install sentence-transformers

     |████████████████████████████████| 78 kB 5.0 MB/s 
     |████████████████████████████████| 3.1 MB 13.8 MB/s 
     |████████████████████████████████| 3.3 MB 29.0 MB/s 
     |████████████████████████████████| 1.2 MB 37.6 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 596 kB 40.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=7d400a8f69a92e558724fc4764f68720b7a2f59f0d84ee79e89044522f4c0978
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [23]:
!pip install nmslib

     |████████████████████████████████| 13.5 MB 29.4 MB/s 
     |████████████████████████████████| 188 kB 48.4 MB/s 


In [24]:
import numpy as np
import pandas as pd
import re
import time
import datasketch
from datasketch import MinHash, MinHashLSHForest
import faiss
import pickle
import nmslib

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

%matplotlib inline

Mounted at /content/gdrive/


In [7]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

# LSH
>  LSH is a hashing based algorithm to identify approximate nearest neighbors. LSH generates a hash value for a data item embeddings while keeping spatiality of data in mind; in particular; data items that are similar in high-dimension will have a higher chance of receiving the same hash value.

Dataset: [Indian Food Receipts](https://www.kaggle.com/nehaprabhavalkar/indian-food-101)

Task: Recommendation Engine of Similar Dishes based on ingredients

In [20]:
df_indian_food = pd.read_csv('/content/gdrive/MyDrive/DataMining/indian_food.csv')

In [21]:
df_indian_food.head(1)

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East


## Shingles
Convert the query text to shingles (tokens)

In [23]:
def preprocessIngredients(text):
  text = text.lower()
  tokens = text.split(', ')
  tokens = list(filter(lambda x: len(x) > 0, tokens))
  return tokens

In [24]:
preprocessIngredients('Maida flour, yogurt, oil, sugar')

['maida flour', 'yogurt', 'oil', 'sugar']

## MinHash and LSH
Apply MinHash and LSH to the shingle set, which maps it to a specific bucket

In [8]:
#Number of permutations
permutations = 128#@param
#Number of Recommendations to return
num_recommendations = 5#@param

In [25]:
df_indian_food.shape

(255, 9)

In [42]:
def create_forest(data, perms, label):
    start_time = time.time()
    minhash = []
    for text in data[label]:
        # process each item in the dataset and convert it to an array of shingles
        tokens = preprocessIngredients(text)
        # set number of permutations in MinHash
        m = MinHash(num_perm=perms)
        # MinHash the string, i.e add to MinHash each shingle in it
        for s in tokens:
            m.update(s.encode('utf8'))
        # store MinHash of the string in the array
        minhash.append(m)
    # build a forest of all MinHash values computed during the previous step 
    forest = MinHashLSHForest(num_perm=perms)
    # index the forest to make it searchable
    for i,m in enumerate(minhash):
        forest.add(i,m)  
    forest.index()
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    return forest

In [43]:
forest = create_forest(df_indian_food, permutations, 'ingredients')

It took 0.38170933723449707 seconds to build forest.


## Similarity search
Conduct a similarity search between the query item and the other items in the bucket

In [44]:
def get_recommendations(text, database, labels, perms, num_results, forest):
    start_time = time.time()
    # tokenize input text 
    tokens = preprocessIngredients(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8')) 
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    result = database.loc[idx_array, labels]
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    return result

In [52]:
ingredients = 'sugar, milk'
result = get_recommendations(ingredients, df_indian_food, ['name', 'ingredients'], permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.008231878280639648 seconds to query forest.

 Top Recommendation(s) is(are) 
               name                  ingredients
8         Kalakand  Milk, cottage cheese, sugar
11           Lassi    Yogurt, milk, nuts, sugar
43  Kakinada khaja           Wheat flour, sugar
21   Chhena kheeri          Chhena, sugar, milk
56         Basundi            Sugar, milk, nuts


# Exhaustive search

> Exhaustive search- Comparing each point to every other point, which will require Linear query time (the size of the dataset)

Dataset: [News Headlines Dataset For Sarcasm Detection](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection?select=Sarcasm_Headlines_Dataset_v2.json)

Task: Find Similar Articles' Headlines

### Get data

In [58]:
df_headlines = pd.read_json('/content/gdrive/MyDrive/DataMining/Sarcasm_Headlines_Dataset_v2.json', lines=True)

In [59]:
df_headlines.head(1)

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...


In [60]:
df_headlines.shape

(28619, 3)

### Apply embeddings using SentenceTransformer

In [61]:
df_headlines['embeddings'] = df_headlines['headline'].apply(lambda x: model.encode(x))

In [129]:
with open('/content/gdrive/MyDrive/DataMining/headlines.pickle', 'wb') as f:
    pickle.dump(df_headlines, f)

In [90]:
def load_data():
    with open('/content/gdrive/MyDrive/DataMining/headlines.pickle', 'rb') as f:
        data = pickle.load(f)
    return data
data = load_data()

### Create a class

In [124]:
class ExactIndex():
    def __init__(self, embeddings, labels):
        embeddings = np.array([embedding for embedding in embeddings])
        self.dimension = embeddings.shape[1]
        self.vectors = embeddings.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, embeddings, k=10):
        embeddings = np.array([embedding for embedding in embeddings])
        distances, indices = self.index.search(embeddings, k) 
        return [self.labels[i] for i in indices[0]]

In [125]:
index = ExactIndex(df_headlines["embeddings"], df_headlines["headline"])
index.build()

### Query dataset

In [127]:
article_vector, article_name = df_headlines["embeddings"][90:91], df_headlines["headline"][90]
index.query(article_vector)
simlar_articles_names = '\n* '.join(index.query(article_vector))
print(f"The most similar movies to {article_name} are:\n* {simlar_articles_names}")

The most similar movies to banksy returns to new york city with one of his trademark rats are:
* banksy returns to new york city with one of his trademark rats
* new documentary makes the case for supervised heroin injection sites in new york
* tony hale reveals the new york inspiration for buster bluth's personality
* giuliani spotted sleeping on new york city subway
* russell simmons leads 'i am a muslim too' rally in new york
* find lead paint violations in new york city neighborhoods
* watch live: bernin up nyc dance party in brooklyn, new york
* congressman says new york city gunman got 'raw deal'
* george and amal cloney are a vision in nyc
* kasich hastily paints name on side of skyscraper in attempt to woo new york voters


# Product Quantization

# Trees and Graphs

# Hierarchical Navigable Small World Graphs

> HNSW grapgh is a proximity graph, in which two vertices are linked based on their proximity (closer vertices are linked) — often defined in Euclidean distance.

Dataset: [Movielens](https://www.kaggle.com/grouplens/movielens-20m-dataset?select=movie.csv)

### Get data

In [8]:
df_movie = pd.read_csv('/content/gdrive/MyDrive/DataMining/movie.csv')

In [9]:
df_movie.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [10]:
df_movie.shape

(27278, 3)

In [16]:
def processGenres(text):
  tags = text.split('|')
  return (' ').join(tags).lower()

In [17]:
processGenres('Adventure|Animation|Children|Comedy|Fantasy')

'adventure animation children comedy fantasy'

In [18]:
df_movie['genres_processed'] = df_movie['genres'].apply(lambda x: processGenres(x))

In [19]:
df_movie.head(1)

,movieId,title,genres,genres_processed
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,adventure animation children comedy fantasy


### Apply embeddings using SentenceTransformer

In [20]:
df_movie['embeddings'] = df_movie['genres_processed'].apply(lambda x: model.encode(x))

In [21]:
with open('/content/gdrive/MyDrive/DataMining/movies_with_embeddings.pickle', 'wb') as f:
    pickle.dump(df_movie, f)

### Create a class

In [26]:
class NMSLIBIndex():
    def __init__(self, embeddings, labels):
        embeddings = np.array([embedding for embedding in embeddings])
        self.vectors = embeddings.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})      
    def query(self, embeddings, k=10):
        embeddings = np.array([embedding for embedding in embeddings])
        indices = self.index.knnQuery(embeddings, k=k)
        return [self.labels[i] for i in indices[0]]

In [27]:
index = NMSLIBIndex(df_movie["embeddings"], df_movie["title"])
index.build()

### Query dataset

In [38]:
movie_index = 30#@param

In [39]:
movie_vector = df_movie["embeddings"][movie_index]
movie_title = df_movie["title"][movie_index]

In [40]:
print(f"Similar movies for {movie_title}")
index.query(movie_vector)

Similar movies for Dangerous Minds (1995)


['Cry, the Beloved Country (1995)',
 'Is Anybody There? (2008)',
 'Little Lord Fauntleroy (1936)',
 "Matthew's Days (1968)",
 'The End of the Tour (2015)',
 'The Old Maid (1939)',
 'Babylon XX (1979)',
 'The Sea Vultures (1915)',
 'The Monastery of Sendomir (1920)',
 'Night Editor (1946)']

# Unused yet

Dataset: [Million Headlines](https://www.kaggle.com/therohk/million-headlines?select=abcnews-date-text.csv)

In [128]:
data = pd.read_csv('/content/gdrive/MyDrive/DataMining/abcnews-date-text.csv')

In [5]:
data.head(1)

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...


In [6]:
def preprocess(text):
    text = text.lower()
    tokens = text.split(' ')
    tokens = list(filter(lambda x: len(x) > 0, tokens))
    return tokens

In [7]:
preprocess(' this IS a test string with  With extra whitespaces ')

['this', 'is', 'a', 'test', 'string', 'with', 'with', 'extra', 'whitespaces']

# References

* [Finding similar images using Deep learning and Locality Sensitive Hashing](https://towardsdatascience.com/finding-similar-images-using-deep-learning-and-locality-sensitive-hashing-9528afee02f5)
* [Implementing LSH in Python](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/)
* [Lacality Sensitive Hashing](https://github.com/santhoshhari/Locality-Sensitive-Hashing)
* [Faiss: A library for efficient similarity search](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)
* [SiameseBERT-SemanticSearch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/SiameseBERT_SemanticSearch.ipynb#scrollTo=AmxRYxNDvn6y)
* [Hierarchical Navigable Small Worlds (HNSW)](https://www.pinecone.io/learn/hnsw/)